In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as snsç
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

In [2]:
from sqlalchemy import create_engine
import pandas as pd

HOST="192.53.165.168"
DBNAME="taxisdb"
USER="airflow"
PASS="airflow"
BATCH_SIZE_TRIP=100000
DATABASE_LOCATION=f'postgresql+psycopg2://{USER}:{PASS}@{HOST}:5432/{DBNAME}'

query="""
SELECT
    id_rate ,
    trip_distance,
    tiempo_viaje,
    pu_b.id_borough as pu_borough,
    do_b.id_borough as do_borough,
    fare_amount
FROM trip
JOIN location pu_l on pu_l.id_location = trip.pu_location
JOIN location do_l on do_l.id_location = trip.do_location
JOIN borough pu_b on pu_b.id_borough = pu_l.id_borough
JOIN borough do_b on do_b.id_borough = do_l.id_borough
"""

In [3]:
engine= create_engine(DATABASE_LOCATION)
with engine.connect()  as conn:
    df_trip=pd.read_sql(query,conn)
df_trip.head()

,id_rate,trip_distance,tiempo_viaje,pu_borough,do_borough,fare_amount
0,1,2.3,1078.0,1,1,12.5
1,1,3.8,1789.0,1,1,19.5
2,1,0.7,293.0,1,1,5.0
3,1,2.8,1176.0,1,1,13.5
4,1,1.3,748.0,1,1,9.0


In [4]:
corr = df_trip.corr()  #is used for find corelation
plt.figure(figsize=(12,12))
sns.heatmap(corr, cbar = True,  square = True, annot=True, fmt= '.2f',annot_kws={'size': 15},
           cmap= 'coolwarm')
plt.xticks(rotation = 90)
plt.yticks(rotation = 45)
plt.show()

NameError: name 'sns' is not defined

<Figure size 864x864 with 0 Axes>

### Arbol de desicion

In [16]:
df_trip

,id_rate,trip_distance,tiempo_viaje,pu_borough,do_borough,fare_amount
0,1,2.3,1078.0,1,1,12.5
1,1,3.8,1789.0,1,1,19.5
2,1,0.7,293.0,1,1,5.0
3,1,2.8,1176.0,1,1,13.5
4,1,1.3,748.0,1,1,9.0
...,...,...,...,...,...,...
26092492,1,1.1,1049.0,1,1,11.5
26092493,1,2.0,976.0,1,1,11.5
26092494,1,0.6,352.0,1,1,5.5
26092495,1,1.3,565.0,1,1,8.0


In [17]:
x_train, x_test, y_train, y_test= train_test_split(df_trip[["id_rate","trip_distance","pu_borough","do_borough"]],df_trip["fare_amount"],test_size=0.2)
x_train

,id_rate,trip_distance,pu_borough,do_borough
11498097,1,0.74,1,1
22967729,1,1.56,1,1
22917381,1,1.92,1,1
23156720,1,1.90,1,1
2148689,2,22.30,4,1
...,...,...,...,...
5147111,1,6.70,1,1
17558056,1,0.40,1,1
11790256,1,3.59,1,1
1787392,1,1.70,1,1


In [18]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer
import numpy as np

In [19]:
from sklearn.model_selection import GridSearchCV
#parameters = {'min_samples_split':[1000],'min_samples_split':[1000,10000],'max_depth':[3]}
parameters = {'min_samples_split':[200,500,1000],'min_samples_leaf':[200,500,1000],'max_depth':[10,20,30]}

In [13]:
regtree = DecisionTreeRegressor()

In [73]:
clf = GridSearchCV(regtree, parameters)
clf.fit(x_train,y_train)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [10, 20, 30],
                         'min_samples_leaf': [200, 500, 1000],
                         'min_samples_split': [200, 500, 1000]})

In [74]:
clf.best_params_

{'max_depth': 20, 'min_samples_leaf': 1000, 'min_samples_split': 200}

In [20]:
regtree = DecisionTreeRegressor(min_samples_split=200, min_samples_leaf=1000, max_depth=20)
regtree.fit(x_train,y_train)
regtree.score(x_test,y_test)

0.9266221489766828

In [95]:
cv = KFold(n_splits = 50, shuffle=True, random_state=1)
cv.get_n_splits(x_train)
scores = cross_val_score(regtree, x_train, y_train, cv = cv, n_jobs=5)
print(scores)
score = np.mean(scores)
print(score)

[9.36208381e-01 9.33954823e-01 9.33433472e-01 9.41031395e-01
 9.17634150e-01 9.06942801e-04 9.04716176e-01 9.37761016e-01
 9.32367866e-01 8.00005399e-01 7.72817240e-01 9.05258332e-01
 9.27757730e-01 9.25862494e-01 9.27414492e-01 9.18319156e-01
 7.92947617e-04 8.98232105e-01 9.34017245e-01 9.24407287e-01
 9.32880833e-01 6.85546346e-01 7.93786967e-01 9.14585418e-01
 9.34234442e-01 9.14786091e-01 9.33167743e-01 9.14946402e-01
 9.22791815e-01 9.32589950e-01 9.28580710e-01 9.17357849e-01
 9.38908826e-01 7.88877862e-01 9.35062233e-01 9.23266773e-01
 7.74598585e-01 9.38115550e-01 9.31087276e-01 9.28595646e-01
 9.38099277e-01 9.34856765e-01 9.26376330e-01 9.32836703e-01
 9.21077798e-01 9.29613145e-01 7.93276385e-01 9.36912323e-01
 9.21029351e-01 7.66214160e-01]
0.8651391640840831


In [15]:
from sklearn.metrics import mean_squared_error
y_train_pred = regtree.predict(x_train)
y_test_pred = regtree.predict(x_test)
print('Error en datos de train:', mean_squared_error(y_train, y_train_pred))
print('Error en datos de test:', mean_squared_error(y_test, y_test_pred))

Error en datos de train: 2219.3557079409306
Error en datos de test: 7679.429523458934


In [111]:
X_new=[[1, 2.0,1,1]]
regtree.predict(X_new)

c:\Users\Nacho\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([10.55818523])

In [22]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle

In [23]:
filename = 'finalized_regtree.sav'
pickle.dump(regtree, open(filename, 'wb'))
 
# some time later...


In [24]:
 
# load the regtree from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [25]:
X_new=[[1, 2.0,1,1]]
loaded_model.predict(X_new)

c:\Users\Nacho\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([10.55627443])